<a href="https://colab.research.google.com/github/saumanraaj/Mycaptain_SaumanRaaj/blob/main/webscraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
  # web scraping using beautifulsoup and requests
  import pandas
  import requests
  import databases 
  import argparse

  parser=argparse.ArgumentParser()
  parser.add_argument("--page_num_Max", help="enter the number of pages to parse", type=int)
  parser.add_argument("--dbname",help="enter the pages toparse",type=int)
  args=parser.parse_args()
  from bs4 import BeautifulSoup
  oyo_url="https://www.oyorooms.com/hotels-in-bangalore/?page="
  page_num_max= args.page_num_Max
  databases.connect(args.dbname)
  for page_num in range(1,page_num_max):
    req= requests.get(oyo_url+ str(page_num)) #getting the information from this url
    content= req.content #accecsing req object from class requests to get the content of the url
    print(content)
    scraped_info_list=[]

  

    soup= BeautifulSoup(content,"html.parser") #creating an object called soup from beautifulsoup cls to parse the html content
    all_hotels= soup.find_all("div",{"class":"hotelCardListing"}) #finding all the html code with tag div attribute class and name hotelcardlisting so that u can get the general strucuture for all hotel data list
    #all_hotels is basically a list which returns value which satisfy soup's findall condition

    for hotels in all_hotels:
      hotel_dict={} #creating a dictionary to store all parsed values from the website
      hotel_dict["name"]= hotel.find("h3",{"class":"listingHotelDescription__hotelName"}).txt #getting the name of hotels from the all_hotel's list
      hotel_dict["address"]=hotel.find("span",{"itemprop":"streetAdress"}).txt #getting the address for hotels 
      hotel_dict["price"]=hotel.find("span",{"class":"listingPrice__finalPrice"}).txt #getting hotel prices
      #try except block: here there might be new hotels without ratings so when we parse them we get error 'AttributeError' to avoid that we ask the compiler to exceute inspite of it using try and except
      try:
        hotel_dict["rating"]=hotel.find("span",{"class":"hotelRating__ratingSummary"}).txt #getting hotel reviews
      except AttributeError: #this try while excecuting throws the error 'AttributError' as the hotel rating is not present but the except block catches it and pass makes the code to un inspite of the error
        hotel_dict["raing"]= None
      amnity_list=[]
      parent_amnities= hotel.find("div",{"class":"amnityWrapper"}) #getting the overall amnities offered by the hotel
      for amnity in parent_amnities.find_all("dive",{"class":"amnityWrapper__amnity"}): #stripping down individual amnities and storing them in a list 
        amnity_list.append(amnity.find("span",{"class":"d-body-sm d-textEllipsis"}).text.strip())
      hotel_dict["amnities"]=', '.join(amnity_list[:-1]) #takes the list'amnity_list' and join it's elements using ,  in between them #here u are removing the unwnated last element(junk) from ur amnities list
      
      scraped_info_list.append(hotel_dict) #storing the dictionary as list to help store info as csv file
      databases.insert_into_table(args.dbname, tuple(hotel_dict.values()))#storing values in a table in sql database

    #dataframe= pandas.DataFrame(scraped_info_list) this data frame is actually a data structure used by pandas to store data
    #dataframe.to_csv("oyo.csv")converting data to csv file named oyo
    databases.get_hotel_info(args.dbname)

ModuleNotFoundError: ignored